In [36]:
from label_studio_sdk import Client
import pandas as pd
import collections

# update these variables based on what you see in the terminal when starting label studio
LABEL_STUDIO_URL = 'http://localhost:8883'
API_KEY = '061663eb1d5f2e6ff2fb830326e7173c538c4817'

In [2]:
target_project = 'Testimport_rois_20221102_102108'

predictions_to_import = [
    ('20220805/20220805_0800/20220805_080714.664.0.png', 'Noctiluca' ,),
    ('20220623/20220623_0700/20220623_071103.834.1.png', 'Copepods')
]

In [8]:
csv_file = "../data/rois_20221107_135029.csv"
df = pd.read_csv(csv_file, delimiter=";")
df.head()

,file,prediction,p_Blurry,p_Bubbles,p_Copepods,p_Ctenophore,p_Diatom chain,p_Dinoflagellate,p_Marine snow,p_Noctiluca,p_Other,p_Pluteus larvae
0,/gpfs/work/vaswani/LPcruises/rois/20220623/202...,Noctiluca,0.000264,0.000007,0.000007,0.000007,1.544579e-06,0.000002,0.000004,0.999679,0.000019,0.000010
1,/gpfs/work/vaswani/LPcruises/rois/20220623/202...,Blurry,0.863435,0.001758,0.002701,0.001616,6.855118e-03,0.000896,0.006884,0.107287,0.007222,0.001347
2,/gpfs/work/vaswani/LPcruises/rois/20220623/202...,Marine snow,0.048235,0.000015,0.000102,0.000013,2.409353e-04,0.000008,0.950067,0.001079,0.000228,0.000011
3,/gpfs/work/vaswani/LPcruises/rois/20220623/202...,Noctiluca,0.000001,0.000005,0.000002,0.000004,7.838775e-07,0.000002,0.000001,0.999958,0.000013,0.000012
4,/gpfs/work/vaswani/LPcruises/rois/20220623/202...,Noctiluca,0.000437,0.000192,0.000173,0.000164,8.351880e-05,0.000085,0.000357,0.997755,0.000464,0.000289


In [20]:
p_cols = [x for x in df.columns if x.startswith("p_")]
p_cols

['p_Blurry',
 'p_Bubbles',
 'p_Copepods',
 'p_Ctenophore',
 'p_Diatom chain',
 'p_Dinoflagellate',
 'p_Marine snow',
 'p_Noctiluca',
 'p_Other',
 'p_Pluteus larvae']

In [67]:
df_relevant = df[["file", "prediction"]]

predictions_to_import = []
p_cols = [x for x in df.columns if x.startswith("p_")]

n_top = 3

for idx, line in df.iterrows():
    value_dict = {line[p_col]: p_col for p_col in p_cols}
    orderd_dict = collections.OrderedDict(sorted(value_dict.items(), reverse=True))
    orderd_dict
    predictions_to_import.append(
        (
            line["file"].split("rois/")[-1], 
            line["prediction"], 
            [(f"{col_name}={p_value:.02f}") for i, (p_value, col_name) in enumerate(orderd_dict.items()) if i<=n_top-1 for x in ]
        )
    )

NameError: name 'col_name' is not defined

In [66]:
predictions_to_import[:20]

[('20220623/20220623_0700/20220623_071103.834.1.png',
  'Noctiluca',
  ['p_Noctiluca=1.00', 'p_Blurry=0.00', 'p_Other=0.00']),
 ('20220623/20220623_0700/20220623_074755.195.0.png',
  'Blurry',
  ['p_Blurry=0.86', 'p_Noctiluca=0.11', 'p_Other=0.01']),
 ('20220623/20220623_0700/20220623_073445.339.0.png',
  'Marine snow',
  ['p_Marine snow=0.95', 'p_Blurry=0.05', 'p_Noctiluca=0.00']),
 ('20220623/20220623_0700/20220623_072507.641.0.png',
  'Noctiluca',
  ['p_Noctiluca=1.00', 'p_Other=0.00', 'p_Pluteus larvae=0.00']),
 ('20220623/20220623_0700/20220623_074655.650.1.png',
  'Noctiluca',
  ['p_Noctiluca=1.00', 'p_Other=0.00', 'p_Blurry=0.00']),
 ('20220623/20220623_0700/20220623_075059.894.0.png',
  'Blurry',
  ['p_Blurry=0.81', 'p_Noctiluca=0.19', 'p_Marine snow=0.00']),
 ('20220623/20220623_0700/20220623_074329.732.0.png',
  'Marine snow',
  ['p_Marine snow=0.41', 'p_Blurry=0.39', 'p_Noctiluca=0.04']),
 ('20220623/20220623_0700/20220623_074354.951.0.png',
  'Noctiluca',
  ['p_Noctiluca=1.

In [3]:
ls = Client(url=LABEL_STUDIO_URL, api_key=API_KEY)

In [4]:
all_projects = ls.list_projects()

In [5]:
project_names = [p.get_params()['title'] for p in all_projects]
print(project_names)

['Testimport_rois_20221102_102108', 'Phytodive_CNN_eval', 'Helgoland_2021_2', 'Phytodive_CPICS']


In [6]:
project_idx = project_names.index(target_project)

In [7]:
project = all_projects[project_idx]

In [8]:
tasks = project.get_tasks()  # for large projects, might need to specify a filter here or it will be slow

In [9]:
images = [task['data']['image'] for task in tasks]

# remove server info so we just have the relative filename
for j, s in enumerate(images):
    if ':' not in s:
        continue
    last_colon = max(loc for loc, val in enumerate(s) if val == ':')
    images[j] = s[last_colon + 6:]
    
print(images)

['20220805/20220805_0800/20220805_080714.664.0.png', '20220609/20220609_1000/20220609_102524.052.0.png', '20220623/20220623_0700/20220623_071103.834.1.png', '20220623/20220623_0700/20220623_071103.834.1.png']


In [21]:
predictions = []
for i, (img, label) in enumerate(predictions_to_import):
    task_idx = images.index(img)
    predictions.append({
        "task": tasks[task_idx]['id'],
        "result": label
    })

In [24]:
predictions

[{'task': 1091170, 'result': 'Noctiluca'},
 {'task': 1091172, 'result': 'Copepods'}]

In [23]:
project.create_predictions(predictions)

{'created': 2}